# Etape 1

In [4]:
import pylab as pl
import pandas as pd

T = pd.read_csv('./data/season-1819_csv.csv')

col = ['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG']
T = T[col]

T.rename(columns = {'HomeTeam' : 'DOM', 'AwayTeam' : 'EXT', 'FTHG' : 'BDOM', 'FTAG' : 'BEXT'},
         inplace = True)

T.head()

,Date,DOM,EXT,BDOM,BEXT
0,10/08/2018,Marseille,Toulouse,4,0
1,11/08/2018,Angers,Nimes,3,4
2,11/08/2018,Lille,Rennes,3,1
3,11/08/2018,Montpellier,Dijon,1,2
4,11/08/2018,Nantes,Monaco,1,3


In [5]:
len(T)

380